# Rechtspraak, LI & citations to DynamoDB ETL

In [ ]:
!pip install boto3

In [1]:
import boto3
from boto3.dynamodb.conditions import Key
from decimal import Decimal
import json
import csv
from pprint import pprint
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np

---
## Raw Datasets

In [2]:
pd.set_option('display.max_columns', None)
df_case_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case.csv", nrows=500000).replace(np.nan, 'NULL', regex=True)
df_case_raw.head()

/home/maxine/anaconda3/envs/case-law-explorer/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,subject,authority,legal_references,related_cases,alternative_sources,full_textcase_id,date.1,case_number.1,description.1,language.1,venue.1,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:GHAMS:1913:15,1913-03-31,Onbekend,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Gerechtshof Amsterdam,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,ECLI:NL:GHAMS:1913:28,1913-02-28,Onbekend389,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Gerechtshof Amsterdam,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,ECLI:NL:GHAMS:1913:19,1913-03-17,Onbekend,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Gerechtshof Amsterdam,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,ECLI:NL:GHAMS:1913:4,1913-02-28,Onbekend,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Gerechtshof Amsterdam,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,ECLI:NL:GHAMS:1913:20,1913-03-10,Onbekend,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Gerechtshof Amsterdam,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [3]:
df_country_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/countries.csv").replace(np.nan, 'NULL', regex=True)
df_country_raw.head()

,country_id,eu,name,flag
0,AT,1,Austria,https://www.countryflags.io/at/flat/64.png
1,BE,1,Belgium,https://www.countryflags.io/be/flat/64.png
2,BG,1,Bulgaria,https://www.countryflags.io/bg/flat/64.png
3,CH,0,Switzerland,https://www.countryflags.io/ch/flat/64.png
4,CZ,1,Czechia,https://www.countryflags.io/cz/flat/64.png


In [4]:
df_li_cases_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legal_intelligence_cases.csv").replace(np.nan, 'NULL', regex=True).drop_duplicates(subset ="ecli")
df_li_cases_raw.head()

,Unnamed: 0,IssueNumber,Id,Title,DisplayTitle,DisplaySubtitle,abstract,Url,OriginalUrl,Jurisdiction,DocumentType,authority,case_number,date,lodge_date,DateAdded,Sources,UrlWithAutoLogOnToken,LawArea,ecli,PublicationNumber
0,0,NULL,4968258,Casus Bij besluit van 12 april 1994 verlenen g...,Casus Bij besluit van 12 april 1994 verlenen g...,"Doorduijn, L.F. - Milieurecht Totaal - Module-...",Casus Bij besluit van 12 april 1994 verlenen g...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da785f3d8d...,Nederland,Rechtspraak,Raad van State,E03951413,19960101,19960916,20110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_AP7727,NULL
1,1,NULL,4968759,M.N. Boeve Casus GS van Noord-Brabant hebben (...,M.N. Boeve Casus GS van Noord-Brabant hebben (...,"Boeve, M.N. - Milieurecht Totaal - Module-seri...",M.N. Boeve Casus GS van Noord-Brabant hebben (...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dae9667789...,Nederland,Rechtspraak,Raad van State,E01.94.0417 ECLI:NL:RVS:1996:AP7883,19960101,19960708,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_AP7883,NULL
2,2,NULL,4968382,Naast woning gelegen huisje vormt ┌┌n geheel m...,Naast woning gelegen huisje vormt ┌┌n geheel m...,Redactie - Module-serie - Module Afval - Milie...,Naast woning gelegen huisje vormt ┌┌n geheel m...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da7b16fb16...,Nederland,Rechtspraak,Gerechtshof Amsterdam,95/4927-E,19960101,19961203,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_GHAMS_1996_AS4281,NULL
3,3,NULL,4968678,Geen beroep mogelijk tegen weigering bestemmin...,Geen beroep mogelijk tegen weigering bestemmin...,Redactie - Module-serie - Module Ruimtelijke O...,De raad van de gemeente Waspik wees een verzoe...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dad97422ad...,Nederland,Rechtspraak,Raad van State,H01950354,19960101,19960201,20110628,Module Ruimtelijke Ordening,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_ZF1978,NULL
4,4,NULL,4968777,Bodemverontreiniging; saneringskosten komen oo...,Bodemverontreiniging; saneringskosten komen oo...,"Kruimel, J.P. - Milieurecht Totaal",Bodemverontreiniging; saneringskosten komen oo...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/daf99e49b6...,Nederland,Rechtspraak,Gerechtshof Den Haag,95/1803,19960101,19961016,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_GHSGR_1996_AS3490,NULL


In [5]:
df_case_opinion_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case_opinion_from_advocate_general.csv").replace(np.nan, 'NULL', regex=True)
df_case_opinion_raw.head()

/home/maxine/anaconda3/envs/case-law-explorer/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,subject,authority,legal_references,related_cases,alternative_sources,full_textcase_id,date.1,case_number.1,description.1,language.1,venue.1,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:PHR:1913:AG1763,1913-04-18,x47,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Parket bij de Hoge Raad,NULL,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1763\n,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,ECLI:NL:PHR:1913:AG1764,1913-04-18,x48,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Parket bij de Hoge Raad,NULL,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1764\n,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,ECLI:NL:PHR:1913:BG9429,1913-04-21,NULL,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Parket bij de Hoge Raad,NULL,Arrest Hoge Raad: ECLI:NL:HR:1913:BG9429\n,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,ECLI:NL:PHR:1913:AG1832,1913-03-07,X46,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Parket bij de Hoge Raad,NULL,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1832\n,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,ECLI:NL:PHR:1913:AG1762,1913-06-20,x49,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Parket bij de Hoge Raad,NULL,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1762\n,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [6]:
df_case_citation_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/caselaw_citations.csv").replace(np.nan, 'NULL', regex=True)
df_case_citation_raw.head()

,source_ecli,source_paragraph,target_ecli,target_paragraph
0,ECLI:NL:HR:1913:13,NULL,ECLI:NL:HR:1913:129,NULL
1,ECLI:NL:HR:1913:130,NULL,ECLI:NL:HR:1913:129,NULL
2,ECLI:NL:HR:1913:131,NULL,ECLI:NL:HR:1913:129,NULL
3,ECLI:NL:HR:1913:132,NULL,ECLI:NL:HR:1913:129,NULL
4,ECLI:NL:HR:1913:133,NULL,ECLI:NL:HR:1913:129,NULL


In [7]:
df_legislation_citation_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legislation_citations.csv").replace(np.nan, 'NULL', regex=True)
df_legislation_citation_raw.head()


,source_ecli,source_paragraph,target_article,target_article_paragraph,target_article_webpage
0,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...
1,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...
2,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2018-...
3,ECLI:NL:GHAMS:1962:AX8441,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...
4,ECLI:NL:GHAMS:1962:AX8441,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...


### insights raw data


In [8]:
print('number of cases: ', len(df_case_raw))
print('number of opinions: ', len(df_case_opinion_raw))
print('number of li-cases: ', len(df_li_cases_raw))
print('number of legislation citations: ', len(df_legislation_citation_raw))
print('number of case citations: ', len(df_case_citation_raw))
print('number of countries: ', len(df_country_raw))

number of cases:  500000
number of opinions:  65692
number of li-cases:  661
number of legislation citations:  18947
number of case citations:  758268
number of countries:  36


---
## Normalization based on constraints

In [9]:
### ecli formatting:
for i, li_case in df_li_cases_raw.iterrows():
    ecli = li_case.ecli
    new_ecli = ecli.replace('_', ':')
    df_li_cases_raw.at[i, 'ecli'] = new_ecli

In [10]:
''' intersection of cases in li and case_raw '''
li_intersection = list(set(df_li_cases_raw['ecli']).intersection(set(df_case_raw['case_id'])))
rechtspraak_only = list(set(df_case_raw['case_id']) - set(li_intersection))
li_only = list(set(df_li_cases_raw['ecli']) - set(li_intersection))
print('number of li only cases: ', len(li_only))
print('number of rechtspraak only cases: ', len(rechtspraak_only))
print('number of overlapping cases: ', len(li_intersection))
print('number of rechtspraak total cases: ', len(df_case_raw))
print('number of li total cases: ', len(df_li_cases_raw))

number of li only cases:  44
number of rechtspraak only cases:  499383
number of overlapping cases:  617
number of rechtspraak total cases:  500000
number of li total cases:  661


In [11]:
### build separate dfs for li only and raw only cases
df_case_only = df_case_raw[df_case_raw['case_id'].isin(rechtspraak_only)]
df_li_only = df_li_cases_raw[df_li_cases_raw['ecli'].isin(li_only)]

In [12]:
### merge overlapping li and raw cases into one df
df_case_li_intersection = df_li_cases_raw[df_li_cases_raw['ecli'].isin(li_intersection)]
df_case_li_intersection = df_case_li_intersection.add_suffix('_li')
df_case_li_intersection.rename(columns = {'ecli_li':'case_id'}, inplace = True)
df_case_li_intersection.set_index('case_id', inplace=True)
df_case_raw.set_index('case_id', inplace=True)
df_case_li_intersection = df_case_li_intersection.join(df_case_raw)
df_case_raw.reset_index(inplace=True)
df_case_li_intersection.reset_index(inplace=True)
df_case_li_intersection.head()

,case_id,Unnamed: 0_li,IssueNumber_li,Id_li,Title_li,DisplayTitle_li,DisplaySubtitle_li,abstract_li,Url_li,OriginalUrl_li,Jurisdiction_li,DocumentType_li,authority_li,case_number_li,date_li,lodge_date_li,DateAdded_li,Sources_li,UrlWithAutoLogOnToken_li,LawArea_li,PublicationNumber_li,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,subject,authority,legal_references,related_cases,alternative_sources,full_textcase_id,date.1,case_number.1,description.1,language.1,venue.1,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:RVS:1996:AP7727,0,NULL,4968258,Casus Bij besluit van 12 april 1994 verlenen g...,Casus Bij besluit van 12 april 1994 verlenen g...,"Doorduijn, L.F. - Milieurecht Totaal - Module-...",Casus Bij besluit van 12 april 1994 verlenen g...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da785f3d8d...,Nederland,Rechtspraak,Raad van State,E03951413,19960101,19960916,20110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,NULL,1996-09-16,E03951413,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Raad van State,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,ECLI:NL:RVS:1996:AP7883,1,NULL,4968759,M.N. Boeve Casus GS van Noord-Brabant hebben (...,M.N. Boeve Casus GS van Noord-Brabant hebben (...,"Boeve, M.N. - Milieurecht Totaal - Module-seri...",M.N. Boeve Casus GS van Noord-Brabant hebben (...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dae9667789...,Nederland,Rechtspraak,Raad van State,E01.94.0417 ECLI:NL:RVS:1996:AP7883,19960101,19960708,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,NULL,1996-07-08,E01.94.0417,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Raad van State,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,ECLI:NL:GHAMS:1996:AS4281,2,NULL,4968382,Naast woning gelegen huisje vormt ┌┌n geheel m...,Naast woning gelegen huisje vormt ┌┌n geheel m...,Redactie - Module-serie - Module Afval - Milie...,Naast woning gelegen huisje vormt ┌┌n geheel m...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da7b16fb16...,Nederland,Rechtspraak,Gerechtshof Amsterdam,95/4927-E,19960101,19961203,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,NULL,1996-12-03,95/4927,NULL,NL,NULL,NULL,NULL,NULL,NL,NULL,Gerechtshof Amsterdam,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,ECLI:NL:RVS:1996:ZF1978,3,NULL,4968678,Geen beroep mogelijk tegen weigering bestemmin...,Geen beroep mogelijk tegen weigering bestemmin...,Redactie - Module-serie - Module Ruimtelijke O...,De raad van de gemeente Waspik wees een verzoe...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dad97422ad...,Nederland,Rechtspraak,Raad van State,H01950354,19960101,19960201,20110628,Module Ruimtelijke Ordening,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,NULL,1996-02-01,H01.95.0354,NULL,NL,NULL,NULL,Bodemzaak,NULL,NL,Bestuursrecht,Raad van State,NULL,Eerste aanleg: ECLI:NL:RBBRE:1995:ZF4362\n,"\n \n <rdf:list xmlns:rdf=""htt...",NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,ECLI:NL:GHSGR:1996:AS3490,4,NULL,4968777,Bodemverontreiniging; saneringskosten komen oo...,Bodemverontreiniging; saneringskosten komen oo...,"Kruimel, J.P. - Milieurecht Totaal",Bodemverontreiniging; saneringskosten komen oo...,http://www.legalintelligence.com/documents/496...,https:/

In [13]:
print('Number of Cases in\nR only cases: {}\nLI only cases: {}\nLI & R cases: {}\nLegislation citation: {}\nCase citation: {}\nCase opinion: {}'\
      .format(len(df_case_only), len(df_li_only), len(df_case_li_intersection), len(df_legislation_citation_raw), len(df_case_citation_raw), len(df_case_opinion_raw)))


Number of Cases in
R only cases: 499383
LI only cases: 44
LI & R cases: 617
Legislation citation: 18947
Case citation: 758268
Case opinion: 65692


---
### Utils

In [14]:
import string

def clean_strings(column):
  cleaned = []
  for s in column:
    cleaned.append("".join(filter(lambda char: char in string.printable, s)))
  return [i[0:250] for i in cleaned]


---
### Preprocessing
#### Courts

In [15]:
court = pd.DataFrame()
courts_list = list(set(df_case_raw.authority.unique().tolist() + df_case_opinion_raw.authority.unique().tolist() + df_li_cases_raw.authority.unique().tolist()))
print(courts_list)
courts_list = [i.replace('"','-') for i in courts_list]
print(courts_list)
courts_list.extend(['Other'])

court['name'] = clean_strings(courts_list)
court.loc[:,'type'] = 'NULL'
court.loc[:,'level'] = 'NULL'
court.loc[:,'country'] = 'NULL'
court.loc[:,'language'] = 'NULL'
court.loc[:,'jurisdiction'] = 'NULL'
court.loc[:,'law_area'] = 'NULL'
court.loc[:,'authority_level'] = 'NULL'

court.head()

['Codecommissie KOAG/KAG', "Ambtenarengerecht 's-Gravenhage", 'Vaste Commissie voor het Boekenuitgeverijbedrijf', 'Kamer voor Gerechtsdeurwaarders', 'Regionaal Directeur Arbeidsvoorziening Dordrecht', 'Consiglio di Stato (Raad van State)', 'Ostre Landsret (gerechtshof Oost-Denemarken)', 'Staatssecretaris VROM', 'Tribunal administratif de Lyon', 'Hooggerechtshof Batavia', 'B&W Rotterdam', 'Gedelegeerde Rechtsherstel Soerabaja', 'Højesteret', 'Hof van arbitrage', 'Kantongerecht Zevenbergen', 'Minister van Economische Zaken', 'Amtsgericht Niebull (kantongerecht)', 'Commissione tributaria provinciale di Massa Carrarra', 'Raad van beroep Middelburg', 'Tribunale Amministrativo per la Sardegna', 'GS Groningen', "Tribunal d'instance de Saintes", 'Onbekend', 'Geschillencommissie chemische wasserijen', 'Kantongerecht Alkmaar', 'HMG Nederlands-Indië', 'Raad van beroep Leeuwarden', 'Raad van Toezicht Orde van Advocaten bij de Hoge Raad', 'Sector kanton Rechtbank Breda', 'Kantongerecht Bergum', 'Ge

,name,type,level,country,language,jurisdiction,law_area,authority_level
0,Codecommissie KOAG/KAG,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,Ambtenarengerecht 's-Gravenhage,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,Vaste Commissie voor het Boekenuitgeverijbedrijf,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,Kamer voor Gerechtsdeurwaarders,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,Regionaal Directeur Arbeidsvoorziening Dordrecht,NULL,NULL,NULL,NULL,NULL,NULL,NULL


#### Cases only

In [28]:
map_case_only = {'date':'date',
            'description':'description',
            'language':'language',
            'venue':'venue',
            'abstract':'abstract',
            'procedure_type':'procedure_type',
            'lodge_date':'lodge_date',
            'alternative_sources':'link',
            'case_id':'ecli',
            'country':'country',
            'subject':'subject',
            'authority':'court'}

case = df_case_only[map_case_only.keys()].rename(columns=map_case_only)
case['name'] = 'NULL'
#case['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['date']]
#case['lodge_date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['lodge_date']]
case['description'] = clean_strings(case['description'])
case['link'] = clean_strings(case['link'])
case['doctype'] = ['uitspraak'] * len(case)

case.head()

,date,description,language,venue,abstract,procedure_type,lodge_date,link,ecli,country,subject,court,name,doctype
0,1913-03-31,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:15,NL,NULL,Gerechtshof Amsterdam,NULL,uitspraak
1,1913-02-28,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:28,NL,NULL,Gerechtshof Amsterdam,NULL,uitspraak
2,1913-03-17,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:19,NL,NULL,Gerechtshof Amsterdam,NULL,uitspraak
3,1913-02-28,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:4,NL,NULL,Gerechtshof Amsterdam,NULL,uitspraak
4,1913-03-10,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:20,NL,NULL,Gerechtshof Amsterdam,NULL,uitspraak


#### Legal Intelligence only Cases

In [29]:
map_legal_intelligence_only_case = {
    'ecli':'ecli',
    'Title':'name_li',
    'date':'date_li',
    'abstract':'abstract_li',
    'LawArea':'subject_li',
    'Url':'link_li',
    'DisplayTitle':'DisplayTitle_li',
    'OriginalUrl':'OriginalUrl_li',
    'Jurisdiction':'Jurisdiction_li',
    'DocumentType':'DocumentType_li',
    'case_number':'CaseNumber_li',
    'PublicationNumber':'PublicationNumber_li',
    'IssueNumber':'IssueNumber_li',
    'lodge_date':'lodge_date_li',
    'DateAdded':'DateAdded_li',
    'Sources':'Sources_li',
    'UrlWithAutoLogOnToken':'UrlWithAutoLogOnToken_li',
    'authority':'court_li',
    'DisplaySubtitle':'DisplaySubtitle_li'
}

legal_intelligence_case = df_li_only[map_legal_intelligence_only_case.keys()].rename(columns=map_legal_intelligence_only_case)
#legal_intelligence_case['date_li'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_only['date']]
#legal_intelligence_case['lodge_date_li'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_only['lodge_date']]
#legal_intelligence_case['DateAdded_li'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_only['DateAdded']]
legal_intelligence_case['name_li'] = clean_strings(legal_intelligence_case['name_li'])
legal_intelligence_case['abstract_li'] = clean_strings(legal_intelligence_case['abstract_li'])
legal_intelligence_case['DisplayTitle_li'] = clean_strings(legal_intelligence_case['DisplayTitle_li'])
legal_intelligence_case['court_li'] = clean_strings(legal_intelligence_case['court_li'])
legal_intelligence_case['DisplaySubtitle_li'] = clean_strings(legal_intelligence_case['DisplaySubtitle_li'])
legal_intelligence_case['doctype'] = ['uitspraak'] * len(legal_intelligence_case)

legal_intelligence_case.head()

,ecli,name_li,date_li,abstract_li,subject_li,link_li,DisplayTitle_li,OriginalUrl_li,Jurisdiction_li,DocumentType_li,CaseNumber_li,PublicationNumber_li,IssueNumber_li,lodge_date_li,DateAdded_li,Sources_li,UrlWithAutoLogOnToken_li,court_li,DisplaySubtitle_li,doctype
14,ECLI:NL:PHR:1996:AA1795,Bewijslast voor geen gebruik bij belanghebbend...,19960101,Bewijslast voor geen gebruik bij belanghebbend...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,Bewijslast voor geen gebruik bij belanghebbend...,https://www.navigator.nl/#/document/da76d44977...,Nederland,Rechtspraak,31 032 ECLI:NL:HR:1996:AA1795,NULL,NULL,19961218,20110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Hoge Raad,Redactie e.a. - Module-serie - Module Afval - ...,uitspraak
1061,ECLI:NL:PHR:1996:AA1822,"Mr W. de Wit Casus Aan belanghebbende, exploit...",19960101,"mr W. de Wit Casus Aan belanghebbende, exploit...",['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,"Mr W. de Wit Casus Aan belanghebbende, exploit...",https://www.navigator.nl/#/document/daa08e5265...,Nederland,Rechtspraak,29996 ECLI:NL:HR:1996:AA1822,NULL,NULL,19960313,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Hoge Raad,Milieurecht Totaal - Module-serie - Module Water,uitspraak
1113,ECLI:NL:PHR:2001:AB1839,"Smeltslakken, Straalmiddel, Baksteenfabricage",19960101,"Smeltslakken, Straalmiddel, Baksteenfabricage",['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/731...,"JAF 1996/11 - Met noot - Meijden van der, D. -...",NULL,Nederland,Rechtspraak,34368,JAF 1996/11,JAF/1996/000001,20010530,20120904,JAF,http://www.legalintelligence.com/documents/731...,Parket bij de Hoge Raad,"Smeltslakken, Straalmiddel, Baksteenfabricage",uitspraak
1341,ECLI:NL:PHR:1996:AA1778,"INKOMSTENBELASTING Een onwettig, niet-erkend k...",19960101,X woont in Nederland. In geschil is of X in 19...,['Belastingrecht'],http://www.legalintelligence.com/documents/519...,"V-N 1996/4773, 17 - Met noot - Redactie - Hoge...",https://www.navigator.nl/#/document/inodfd8a8c...,Nederland,Rechtspraak,31 357 ECLI:NL:HR:1996:AA1778,"V-N 1996/4773, 17",VN/1996/000060,19961127,20110623,VN,http://www.legalintelligence.com/documents/519...,Hoge Raad,"INKOMSTENBELASTING Een onwettig, niet-erkend k...",uitspraak
1523,ECLI:NL:PHR:1996:AA1790,SUCCESSIEWET Verrekening van schenkingsrecht m...,19960101,Erflaatster schonk effecten aan haar kinderen ...,"['Belastingrecht', 'Onbekend', 'Personen- en F...",http://www.legalintelligence.com/documents/519...,"V-N 1996/4683, 23 - Met noot - Redactie - Hoge...",https://www.navigator.nl/#/document/inodfd991c...,Nederland,Rechtspraak,31 205 ECLI:NL:HR:1996:AA1790,"V-N 1996/4683, 23",VN/1996/000059,19961120,20110623,VN,http://www.legalintelligence.com/documents/519...,Hoge Raad,SUCCESSIEWET Verrekening van schenkingsrecht m...,uitspraak


#### Cases & LI

In [30]:
map_case_li = {'date':'date',
            'description':'description',
            'language':'language',
            'venue':'venue',
            'abstract':'abstract',
            'procedure_type':'procedure_type',
            'lodge_date':'lodge_date',
            'alternative_sources':'link',
            'case_id':'ecli',
            'country':'country',
            'subject':'subject',
            'authority':'court',
            'Title_li':'name_li',
            'date_li':'date_li',
            'abstract_li':'abstract_li',
            'LawArea_li':'subject_li',
            'Url_li':'link_li',
            'DisplayTitle_li':'DisplayTitle_li',
            'OriginalUrl_li':'OriginalUrl_li',
            'Jurisdiction_li':'Jurisdiction_li',
            'DocumentType_li':'DocumentType_li',
            'case_number_li':'CaseNumber_li',
            'PublicationNumber_li':'PublicationNumber_li',
            'IssueNumber_li':'IssueNumber_li',
            'lodge_date_li':'lodge_date_li',
            'DateAdded_li':'DateAdded_li',
            'Sources_li':'Sources_li',
            'UrlWithAutoLogOnToken_li':'UrlWithAutoLogOnToken_li',
            'authority_li':'court_li',
            'DisplaySubtitle_li':'DisplaySubtitle_li'
        }

case_li = df_case_li_intersection[map_case_li.keys()].rename(columns=map_case_li)
case_li['name'] = 'NULL'
#case_li['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case_li['date']]
#case_li['lodge_date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case_li['lodge_date']]
case_li['description'] = clean_strings(case_li['description'])
case_li['link'] = clean_strings(case_li['link'])
case_li = df_case_li_intersection[map_case_li.keys()].rename(columns=map_case_li)
#case_li['date_li'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_case_li_intersection['date_li']]
#case_li['lodge_date_li'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_case_li_intersection['lodge_date_li']]
#case_li['DateAdded_li'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_case_li_intersection['DateAdded_li']]
case_li['name_li'] = clean_strings(case_li['name_li'])
case_li['abstract_li'] = clean_strings(case_li['abstract_li'])
case_li['DisplayTitle_li'] = clean_strings(case_li['DisplayTitle_li'])
case_li['court_li'] = clean_strings(case_li['court_li'])
case_li['DisplaySubtitle_li'] = clean_strings(case_li['DisplaySubtitle_li'])
case_li['doctype'] = ['uitspraak'] * len(case_li)

case_li.head()

,date,description,language,venue,abstract,procedure_type,lodge_date,link,ecli,country,subject,court,name_li,date_li,abstract_li,subject_li,link_li,DisplayTitle_li,OriginalUrl_li,Jurisdiction_li,DocumentType_li,CaseNumber_li,PublicationNumber_li,IssueNumber_li,lodge_date_li,DateAdded_li,Sources_li,UrlWithAutoLogOnToken_li,court_li,DisplaySubtitle_li,doctype
0,1996-09-16,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:RVS:1996:AP7727,NL,NULL,Raad van State,Casus Bij besluit van 12 april 1994 verlenen g...,19960101,Casus Bij besluit van 12 april 1994 verlenen g...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,Casus Bij besluit van 12 april 1994 verlenen g...,https://www.navigator.nl/#/document/da785f3d8d...,Nederland,Rechtspraak,E03951413,NULL,NULL,19960916,20110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Raad van State,"Doorduijn, L.F. - Milieurecht Totaal - Module-...",uitspraak
1,1996-07-08,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:RVS:1996:AP7883,NL,NULL,Raad van State,M.N. Boeve Casus GS van Noord-Brabant hebben (...,19960101,M.N. Boeve Casus GS van Noord-Brabant hebben (...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,M.N. Boeve Casus GS van Noord-Brabant hebben (...,https://www.navigator.nl/#/document/dae9667789...,Nederland,Rechtspraak,E01.94.0417 ECLI:NL:RVS:1996:AP7883,NULL,NULL,19960708,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Raad van State,"Boeve, M.N. - Milieurecht Totaal - Module-seri...",uitspraak
2,1996-12-03,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1996:AS4281,NL,NULL,Gerechtshof Amsterdam,Naast woning gelegen huisje vormt n geheel met...,19960101,Naast woning gelegen huisje vormt n geheel met...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,Naast woning gelegen huisje vormt n geheel met...,https://www.navigator.nl/#/document/da7b16fb16...,Nederland,Rechtspraak,95/4927-E,NULL,NULL,19961203,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Gerechtshof Amsterdam,Redactie - Module-serie - Module Afval - Milie...,uitspraak
3,1996-02-01,NULL,NL,NULL,NULL,Bodemzaak,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:RVS:1996:ZF1978,NL,Bestuursrecht,Raad van State,Geen beroep mogelijk tegen weigering bestemmin...,19960101,De raad van de gemeente Waspik wees een verzoe...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,Geen beroep mogelijk tegen weigering bestemmin...,https://www.navigator.nl/#/document/dad97422ad...,Nederland,Rechtspraak,H01950354,NULL,NULL,19960201,20110628,Module Ruimtelijke Ordening,http://www.legalintelligence.com/documents/496...,Raad van State,Redactie - Module-serie - Module Ruimtelijke O...,uitspraak
4,1996-10-16,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHSGR:1996:AS3490,NL,NULL,Gerechtshof 's-Gravenhage,Bodemverontreiniging; saneringskosten komen oo...,19960101,Bodemverontreiniging; saneringskosten komen oo...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,Bodemverontreiniging; saneringskosten komen oo...,https://www.navigator.nl/#/document/daf99e49b6...,Nederland,Rechtspraak,95/1803,NULL,NULL,19961016,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Gerechtshof Den Haag,"Kruimel, J.P. - Milieurecht Totaal",uitspraak


#### Case opinion advocate general

In [31]:
map_case_opinion = {'date':'date',
                    'case_number':'case_number',
                    'description':'description',
                    'language':'language',
                    'country':'country',
                    'venue':'venue',
                    'abstract':'abstract',
                    'procedure_type':'procedure_type',
                    'authority':'court',
                    'case_id':'ecli',
                    'subject':'subject'}

case_opinion = df_case_opinion_raw[map_case_opinion.keys()].rename(columns=map_case_opinion)
#case_opinion['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case_opinion['date']]
case_opinion['abstract'] = clean_strings(case_opinion['abstract'])
case_opinion['doctype'] = ['opinion'] * len(case_opinion)

case_opinion.head()



,date,case_number,description,language,country,venue,abstract,procedure_type,court,ecli,subject,doctype
0,1913-04-18,x47,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:AG1763,NULL,opinion
1,1913-04-18,x48,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:AG1764,NULL,opinion
2,1913-04-21,NULL,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:BG9429,NULL,opinion
3,1913-03-07,X46,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:AG1832,NULL,opinion
4,1913-06-20,x49,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:AG1762,NULL,opinion


#### Subjects

In [ ]:
# unique_subjects
#

In [23]:
# subjects_as_list = [list(row.split("; ")) for row in df_case.subject]
# unique_subjects = \
#     set(list(
#         pd.core.common\
#             .flatten(subjects_as_list)))
# subject = pd.DataFrame()
# subject['name'] = clean_strings(list(sorted(unique_subjects)))
# subject.loc[:,'standard_name'] = 'NULL'


#### Countries

In [57]:
# df_country.loc[:,'language'] = 'NULL'
# df_country.loc[:,'eea'] = 0
# country = df_country[['country_id','name','language','flag','eu','eea']]\
#   .rename(columns={'country_id':'id'})

#### Case law citation

In [20]:
map_case_citation = {
    'source_ecli':'ecli',
    'source_paragraph':'source_paragraph',
    'target_ecli':'target_ecli',
    'target_paragraph':'target_paragraph'
}
case_citation = df_case_citation_raw[map_case_citation.keys()].rename(columns=map_case_citation)
case_citation['doctype'] = 'c-citation-' + case_citation['target_ecli'].map(str)

case_citation.head()


,ecli,source_paragraph,target_ecli,target_paragraph,doctype
0,ECLI:NL:HR:1913:13,NULL,ECLI:NL:HR:1913:129,NULL,c-citation-ECLI:NL:HR:1913:129
1,ECLI:NL:HR:1913:130,NULL,ECLI:NL:HR:1913:129,NULL,c-citation-ECLI:NL:HR:1913:129
2,ECLI:NL:HR:1913:131,NULL,ECLI:NL:HR:1913:129,NULL,c-citation-ECLI:NL:HR:1913:129
3,ECLI:NL:HR:1913:132,NULL,ECLI:NL:HR:1913:129,NULL,c-citation-ECLI:NL:HR:1913:129
4,ECLI:NL:HR:1913:133,NULL,ECLI:NL:HR:1913:129,NULL,c-citation-ECLI:NL:HR:1913:129


#### Legislation citation

In [21]:
df_legislation_citation_raw.loc[:,'target_name'] = 'NULL'
df_legislation_citation_raw.loc[:,'target_sourcename'] = 'NULL'

map_legislation_citation = {
    'source_ecli':'ecli',
    'source_paragraph':'source_paragraph',
    'target_article':'target_id',
    'target_article_paragraph':'target_paragraph',
    'target_name':'target_name',
    'target_sourcename':'target_sourcename',
    'target_article_webpage':'target_link'
}
legislation_citation = df_legislation_citation_raw[map_legislation_citation.keys()].rename(columns=map_legislation_citation)
legislation_citation['doctype'] = 'l-citation-' + legislation_citation['target_id'].map(str)


legislation_citation.head()

,ecli,source_paragraph,target_id,target_paragraph,target_name,target_sourcename,target_link,doctype
0,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...,l-citation-http://linkeddata.overheid.nl/terms...
1,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...,l-citation-http://linkeddata.overheid.nl/terms...
2,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2018-...,l-citation-http://linkeddata.overheid.nl/terms...
3,ECLI:NL:GHAMS:1962:AX8441,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...,l-citation-http://linkeddata.overheid.nl/terms...
4,ECLI:NL:GHAMS:1962:AX8441,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...,l-citation-http://linkeddata.overheid.nl/terms...


#### Case related decision

In [23]:
df_case_related = df_case_raw[['case_id','related_cases']]\
    .rename(columns = {'case_id':'source_ecli',
                       'related_cases': 'referencing_case_ecli'})
df_case_related.loc[:,'referenced_case_ecli'] = 'NULL'

map_case_related_decision = {
    'source_ecli':'ecli',
    'referencing_case_ecli':'referencing_case_ecli',
    'referenced_case_ecli':'referenced_case_ecli'
    }

case_related_decision = df_case_related[map_case_related_decision.keys()].rename(columns=map_case_related_decision)
case_related_decision['doctype'] = 'related-' + case_related_decision['referencing_case_ecli'].map(str)

case_related_decision.head()

,ecli,referencing_case_ecli,referenced_case_ecli,doctype
0,ECLI:NL:GHAMS:1913:15,NULL,NULL,related-NULL
1,ECLI:NL:GHAMS:1913:28,NULL,NULL,related-NULL
2,ECLI:NL:GHAMS:1913:19,NULL,NULL,related-NULL
3,ECLI:NL:GHAMS:1913:4,NULL,NULL,related-NULL
4,ECLI:NL:GHAMS:1913:20,NULL,NULL,related-NULL


---
## Data to DynamoDB

In [24]:
dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")  # remove 'endpoint_url' to use web service

Table status: ACTIVE


### Table and key schema definitions:

In [ ]:
caselaw_table = dynamodb.create_table(
    TableName='Caselaw',
    KeySchema=[
        {
            'AttributeName': 'ecli',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'doctype',
            'KeyType': 'RANGE'  # Sort key
        },
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'ecli',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'doctype',
            'AttributeType': 'S'
        },
        {
            "AttributeName": "subject",
            "AttributeType": "S"
        },
    ],
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'GSI-1',
            'KeySchema': [
                {
                    'AttributeName': 'doctype',
                    'KeyType': 'HASH'
                },
                                {
                    'AttributeName': 'subject',
                    'KeyType': 'RANGE'
                },
            ],
            'Projection': {
                'ProjectionType': 'ALL',
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 10,
                'WriteCapacityUnits': 10
            },
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)
print("Table status:", caselaw_table.table_status)


### Functions to load data into table (from csv or json):

In [25]:
def load_data(table, data, output=False):
    keys = [key['AttributeName'] for key in table.key_schema]
    counter = 0
    with table.batch_writer() as batch:
        #for item in data:
        for item in data.iterrows():
            item = item[1].to_dict()
            counter += 1
            if output:
                values = [item[key] for key in keys]
                print("Adding item:", dict(zip(keys, values)))
            batch.put_item(Item=item)
    print(f'{counter} items added.')

def read_csv(filename, items):
    data = csv.DictReader(open(filename))
    for item in data:
        items.append(item)

table = dynamodb.Table('caselaw')
#file_name = 'moviedata.csv'

''' from json: '''
#with open("moviedata.json") as json_file:
#    item_list = json.load(json_file, parse_float=str)
''' from csv: '''
item_list = []
#read_csv(file_name, item_list)

In [ ]:
load_data(caselaw_table, case)

In [32]:
load_data(caselaw_table, legal_intelligence_case)

44 items added.


In [34]:
load_data(caselaw_table, case_li)

617 items added.


In [36]:
load_data(caselaw_table, case_opinion)

65692 items added.


In [35]:
load_data(caselaw_table, case_citation)

758268 items added.


In [ ]:
load_data(table, legislation_citation)

In [ ]:
load_data(table, case_related_decision)

### Example queries:

In [37]:
### uitspraak by ecli:
pprint(caselaw_table.get_item(Key={'ecli': 'ECLI:NL:PHR:1996:AA1795',
                                   'doctype': 'uitspraak'}))

{'Item': {'CaseNumber_li': '31 032 ECLI:NL:HR:1996:AA1795',
          'DateAdded_li': Decimal('20110520'),
          'DisplaySubtitle_li': 'Redactie e.a. - Module-serie - Module Afval - '
                                'Milieurecht Totaal',
          'DisplayTitle_li': 'Bewijslast voor geen gebruik bij belanghebbende. '
                             '(Nog) geen oordeel of benutten gesubsidieerde '
                             'organisaties ook gebruik kan zijn van diensten '
                             'vanwege de gemeente',
          'DocumentType_li': 'Rechtspraak',
          'IssueNumber_li': 'NULL',
          'Jurisdiction_li': 'Nederland',
          'OriginalUrl_li': 'https://www.navigator.nl/#/document/da76d4497728a70201a21820933a0c3b04',
          'PublicationNumber_li': 'NULL',
          'Sources_li': 'Milieurecht Totaal',
          'UrlWithAutoLogOnToken_li': 'http://www.legalintelligence.com/documents/4968276?srcfrm=basic+search&app=API&docindex=25&stext=1996&alt=qRFpgllPmW0

In [38]:
### opinion by ecli:
pprint(caselaw_table.get_item(Key={'ecli': 'ECLI:NL:PHR:1913:AG1763',
                                   'doctype': 'opinion'}))

{'Item': {'abstract': 'NULL',
          'case_number': 'x47',
          'country': 'NL',
          'court': 'Parket bij de Hoge Raad',
          'date': '1913-04-18',
          'description': 'NULL',
          'doctype': 'opinion',
          'ecli': 'ECLI:NL:PHR:1913:AG1763',
          'language': 'NL',
          'procedure_type': 'NULL',
          'subject': 'NULL',
          'venue': 'NULL'},
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '335',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'server': 'Jetty(8.1.12.v20130726)',
                                      'x-amz-crc32': '961979117',
                                      'x-amzn-requestid': '66453fd1-28f8-4315-b2c6-51faa7fec4fd'},
                      'HTTPStatusCode': 200,
                      'RequestId': '66453fd1-28f8-4315-b2c6-51faa7fec4fd',
                      'RetryAttempts': 0}}


In [47]:
### case citations by ecli:
pprint(caselaw_table.query(
    KeyConditionExpression=Key('ecli').eq('ECLI:NL:AGAMS:1934:1') &
                           Key('doctype').begins_with('c-citation'))  # [lc, cc, d]
)

{'Count': 6,
 'Items': [{'doctype': 'c-citation-ECLI:NL:HR:1913:129',
            'ecli': 'ECLI:NL:AGAMS:1934:1',
            'source_paragraph': 'NULL',
            'target_ecli': 'ECLI:NL:HR:1913:129',
            'target_paragraph': 'NULL'},
           {'doctype': 'c-citation-ECLI:NL:HR:1914:114',
            'ecli': 'ECLI:NL:AGAMS:1934:1',
            'source_paragraph': 'NULL',
            'target_ecli': 'ECLI:NL:HR:1914:114',
            'target_paragraph': 'NULL'},
           {'doctype': 'c-citation-ECLI:NL:HR:1914:149',
            'ecli': 'ECLI:NL:AGAMS:1934:1',
            'source_paragraph': 'NULL',
            'target_ecli': 'ECLI:NL:HR:1914:149',
            'target_paragraph': 'NULL'},
           {'doctype': 'c-citation-ECLI:NL:HR:1914:157',
            'ecli': 'ECLI:NL:AGAMS:1934:1',
            'source_paragraph': 'NULL',
            'target_ecli': 'ECLI:NL:HR:1914:157',
            'target_paragraph': 'NULL'},
           {'doctype': 'c-citation-ECLI:NL:HR:1915:104',
  

In [46]:
### cases per subject='Bestuursrecht' and court='Hoge Raad'
pprint(caselaw_table.query(
    IndexName='GSI-1',  # only needs to be specified if secondary index is used
    ProjectionExpression="ecli, court, country",
    FilterExpression='court = :a',
    ExpressionAttributeValues={':a': 'Hoge Raad'},
    KeyConditionExpression=Key('doctype').eq('uitspraak') & Key('subject').begins_with('Bestuursrecht'))
)


''' usage example:

pprint(table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={"#yr": "year"},
    KeyConditionExpression=
            Key('year').eq(year) & Key('title').between(title_range[0], title_range[1]))
)
'''

{'Count': 168,
 'Items': [{'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1997:ZF3400'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1997:AA3340'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1997:AA2147'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1998:AA2568'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1998:AA2367'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1999:AA3390'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1996:AA1751'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1998:AA2607'},
           {'country': 'NL',
            'court': 'Hoge Raad',
            'ecli': 'ECLI:NL:HR:1999:AA386